In [1]:
import os
# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import torch

torch.cuda.is_available()

In [3]:
# Data loading
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

# Image size? todo ritornare a 128x128?
full_train_dataset = datasets.ImageFolder("../data/train", transform=transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor()
]))

validation_dataset = datasets.ImageFolder("../data/test/", transform=transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor()
]))

In [4]:
# Pre processing load
from utils import pre_processing

sequential_pre_processing_procedure = pre_processing.AugmentationFactory.make_complete_procedure_keras()

# K-FOLD Loading
We load data and check if functions work as intended

In [5]:
## Naive Model Summary
from models.naive_dnn_gen.naive_dnn import NaiveDnn
from models.simple_cnn.simple_cnn import SimpleCNN
naive_model = SimpleCNN((224, 224, 3), 2, augmentation_procedure=sequential_pre_processing_procedure)

graph = naive_model.build_graph()
graph.summary()

In [5]:
import gc
from k_fold_cv.k_fold_generator import KFoldController
from models.naive_dnn_gen.naive_dnn import NaiveDnn
from models.simple_cnn.simple_cnn import SimpleCNN
from callbacks import learning_rate_reschedule
import keras
from keras import backend

from models.zero_one_validation_loss import zero_one_loss, zero_one_loss_binary

FOLDS = 5

k_fold_controller = KFoldController(FOLDS)
k_fold_controller.load_data(full_train_dataset)

history = []

# todo function
for i in range(FOLDS):
    train, val = k_fold_controller.get_data_for_fold(i)
    train_dataloader = DataLoader(dataset=train, batch_size=32)
    validation_dataloader = DataLoader(dataset=val, batch_size=32)

    naive_model = SimpleCNN((224, 224, 3), augmentation_procedure=sequential_pre_processing_procedure)
    # I moved to Adam as it is faster
    naive_model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.5),
                        metrics=['accuracy', zero_one_loss_binary], loss='binary_crossentropy')
    i_history = naive_model.fit(train_dataloader, epochs=100, validation_data=validation_dataloader,
                                callbacks=[
                                    # To avoid keep going
                                    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                                                  restore_best_weights=True),

                                    # To reduce learning rate (like Hot Stocastic bla bla)
                                    #keras.callbacks.LearningRateScheduler(
                                    #    learning_rate_reschedule.learning_rate_reschedule)
                                ])
    history.append(i_history)

    naive_model.save("k-fold-chihuahua-val-fold-" + str(i) + ".keras")
    naive_model.losses.clear()
    naive_model.metrics.clear()

    # TODO Capisci perche ora va riducendo le chiamate al  minimo indispensabile
    # https://github.com/keras-team/keras/issues/5345
    torch.cuda.empty_cache()
    del naive_model
    torch.cuda.empty_cache()
    gc.collect()
    #gc.collect()  # Python thing

In [ ]:
import pandas

metrics_df1 = pandas.DataFrame(history[0])
metrics_df1[["loss", "val_loss"]].plot()

    http://mipal.snu.ac.kr/images/1/16/Dropout_ACCV2016.pdf
Dropout

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
import matplotlib.pyplot as plt

train_dataloader = DataLoader(datasets.ImageFolder("../data/train/", transform=transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])), batch_size=32, shuffle=True)

validation_dataloader = DataLoader(
    datasets.ImageFolder("../data/test/",
                         transform=transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])),
    batch_size=32, shuffle=True
)

train_features, train_labels = next(iter(train_dataloader))

img = train_features[0].squeeze()
label = train_labels[0]
print(label)
plt.imshow(img.T, cmap="gray")
plt.show()

In [ ]:
from models.simple_neural_network import simple_neural_network

model = simple_neural_network((128, 128, 3), 2)
model.summary()

In [ ]:
model.fit(train_dataloader, epochs=150, validation_data=validation_dataloader)

In [ ]:
from keras.utils import plot_model

plot_model(model, show_shapes=True)